# Анализ запросов на оказание услуг, отправленных через формы обратной связи с сайта за май-октябрь 2020 года

# Содержание

- [Описание проекта](#obj)
- [Выводы](#conclu)
- [Анализ обращений через сайт](#sit)
- [Анализ возможностей](#opp)
- [Анализ конверсий](#con)
- [Воронки коверсий](#fun)

<a id="obj"></a>

# Описание проекта

1. Проанализированы обращения клиентов через сайт: по типу запроса, запрашиваемой услуге, отрасли компании, должности обращающегося 
2. Определен список клиентов, пришедших с сайта по списку возможностей из CRM за тот же период
3. Расчитана помесячная конверсия:
    - посещений в запрос
    - запросов в возможность
    - возможность в выигранную возможность
4. Расчитана воронка конверсий по усредненным данным 

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats as st
import math as mth
import matplotlib.pyplot as plt 
import plotly.express as px
from plotly import graph_objects as go

import texthero as hero
from texthero import preprocessing

import warnings
warnings.simplefilter('ignore')

<a id="sit"></a>

# Анализ обращений через сайт

Таблица с обращениями `rfp_05_09_20.xlsx`

In [ ]:
rfp = pd.read_excel('/Users/Olga/Site_rfp/rfp_05_09_20.xlsx')

In [ ]:
rfp.to_csv('rfp.csv', encoding='utf-8', index=False)

In [ ]:
rfp.info()

In [ ]:
rfp.sample(3)

Преобразуем данные в колонке `Дата изм.` в формат datetime, создадим новую колонку `Дата` из нее выделим отдельный столбец `Месяц` с месяцем обращения

In [ ]:
rfp['Дата'] = pd.to_datetime(rfp['Дата изм.'], format='%d.%m.%Y %H:%M:%S',infer_datetime_format=True)

rfp['Месяц'] = rfp['Дата'].dt.month

### Общее количество обращений в мае-октябре

In [ ]:
pivot_table_by_month_total = rfp.pivot_table(rfp,index=['Месяц'], aggfunc={'ID': 'nunique'}).reset_index()
pivot_table_by_month_total.columns=['Месяц', 'Количество обращений']
pivot_table_by_month_total.sort_values(by = 'Количество обращений', ascending = False).head(10)

### Количество обращений, в среднем

In [ ]:
pivot_table_by_month_total['Количество обращений'].mean()

### Количество обращений в штуках, по месяцам, график

In [ ]:
fig = px.bar(pivot_table_by_month_total, x="Месяц", y="Количество обращений", text="Количество обращений", title="Количество обращений в месяц")
fig.show()

### Наиболее популярный запрос

In [ ]:
rfp['Тема обращения'].unique()

In [ ]:
pivot_table_by_subject = rfp.pivot_table(rfp,index=['Тема обращения'], aggfunc={'ID': 'nunique'}).reset_index()
pivot_table_by_subject.columns=['Тема обращения', 'Количество обращений']
pivot_table_by_subject = pivot_table_by_subject.sort_values(by = 'Количество обращений', ascending = False)


unique = pivot_table_by_subject['Количество обращений'].sum()
pivot_table_by_subject['% от количества обращений'] = ((pivot_table_by_subject['Количество обращений'] / unique)*100)


pivot_table_by_subject.head(10).style.background_gradient(cmap='plasma')

### Наиболее популярная услуга

In [ ]:
pivot_table_by_services = rfp.pivot_table(rfp,index=['Услуга'], aggfunc={'ID': 'nunique'}).reset_index()
pivot_table_by_services.columns=['Услуга', 'Количество обращений']
pivot_table_by_services = pivot_table_by_services.sort_values(by = 'Количество обращений', ascending = False)

pivot_table_by_services.head(10).style.background_gradient(cmap='plasma')

### Интерактивный график: количество обращений в разрезе запрашиваемой услуги 

In [ ]:
fig = px.pie(pivot_table_by_services, values='Количество обращений', names='Услуга', title='Количество обращений в разрезе запрашиваемой услуги')
fig.show()

### Самая частая должность обращающегося: главный бухгалтер

In [ ]:
unique_position = rfp['Должность'].count()
unique_position

In [ ]:
pivot_table_by_position = rfp.pivot_table(rfp,index=['Должность'], aggfunc={'ID': 'nunique'}).reset_index()
pivot_table_by_position.columns=['Должность', 'Количество обращений']

unique_position = rfp['Должность'].count()
pivot_table_by_position['% от количества обращений'] = ((pivot_table_by_position['Количество обращений'] / unique_position )*100)

pivot_table_by_position.sort_values(by = 'Количество обращений', ascending = False).head(10).style.background_gradient(cmap='plasma')


### Список должностей обращающихся через сайт

In [ ]:
rfp['Должность'].unique()

### Количество обращений по услугам в месяц

In [ ]:
pivot_table_by_month = rfp.pivot_table(rfp,index=['Месяц', 'Услуга'], aggfunc={'ID': 'nunique'}).reset_index()
pivot_table_by_month.columns=['Месяц', 'Услуга', 'Количество обращений']
pivot_table_by_month = pivot_table_by_month.sort_values(by = 'Количество обращений', ascending = False)

pivot_table_by_month.head(10).style.background_gradient(cmap='plasma')

#### Помесячный интерактивный график обращений в разрезе запрашиваемой услуги

In [ ]:
fig = px.bar(pivot_table_by_month, x="Месяц", y="Количество обращений", color="Услуга", title="Количество обращений в месяц в разрезе услуг", width=900, height=600)
fig.show()

### Количество обращений по партнерам

In [ ]:
pivot_table_by_person = rfp.pivot_table(rfp,index=['Эксперт'], aggfunc={'ID': 'nunique'}).reset_index()
pivot_table_by_person.columns=['Эксперт', 'Количество обращений']
pivot_table_by_person = pivot_table_by_person.sort_values(by = 'Количество обращений', ascending = False)

pivot_table_by_person.head(10).style.background_gradient(cmap='plasma')

In [ ]:
fig = px.pie(pivot_table_by_person, values='Количество обращений', names='Эксперт', title='Количество обращений по партнерам')
fig.show()


### Количество обращений по отраслям

In [ ]:
pivot_table_by_industries = rfp.pivot_table(rfp,index=['Отрасль'], aggfunc={'ID': 'nunique'}).reset_index()
pivot_table_by_industries.columns=['Отрасль компании', 'Количество обращений']
pivot_table_by_industries = pivot_table_by_industries.sort_values(by = 'Количество обращений', ascending = False)

pivot_table_by_industries.head(5).style.background_gradient(cmap='plasma')

In [ ]:
unique_ID = pivot_table_by_industries['Количество обращений'].sum()
pivot_table_by_industries['% от количества обращений'] = ((pivot_table_by_industries['Количество обращений'] / unique_ID*100).round(2))

In [ ]:
fig = px.pie(pivot_table_by_industries, values='Количество обращений', names='Отрасль компании', title='Количество обращений по отрасли компании')
fig.show()

<a id="opp"></a>
# Анализ возможностей

Загрузим таблицу `opportunities` с данными о новых возможностях из CRM за май-октябрь 2020 года

In [ ]:
opportunities = pd.read_excel('/Users/Olga/First attempt in Py/Site_rfp/opportunities_05_09_20.xlsx')

In [ ]:
opportunities.to_csv('traffic.csv', encoding='utf-8', index=False)

In [ ]:
opportunities.info()

### Возможности по статусу

In [ ]:
opportunities.pivot_table(opportunities,index=['Статус'], aggfunc={'Организация': 'nunique'})

Отсортируем возможности из CRM по вхождению названия компании в столбец `Компания` из данных запросов с сайта. Для этого подготовим столбцы с названиями компаний, очистив данные от знаков, цифр, пунктуации, пробелов.

In [ ]:
custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   , preprocessing.remove_digits
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_whitespace
                   , preprocessing.stem]
                  
rfp['Компания_чистая'] = hero.clean(rfp['Компания'], custom_pipeline)
opportunities['Организация_чистая'] = hero.clean(opportunities['Организация'], custom_pipeline)

Сформируем список компаний, написавших запрос через сайт

In [ ]:
list_companies = rfp['Компания_чистая']

Сделаем срез таблицы с выгруженными из ZOOM возможностями по полученному списку `list_companies`, сравнив вхождение в столбец `Организация_чистая`

In [ ]:
clients = opportunities.query('Организация_чистая in @list_companies')

### Таблица со срезом возможностей, у которых совпадает название компании в обращениях

In [ ]:
clients.to_csv('clients.csv', encoding='utf-8', index=False)

In [ ]:
clients.info()

In [ ]:
clients.head(13)

### Список компаний, пришедших с сайта

In [ ]:
clients['Организация']

### Возможности, пришедших с сайта в разрезе услуг

In [ ]:
pivot_clients = clients.pivot_table(index=['Услуга'], aggfunc={'Организация': 'nunique'})
pivot_clients.columns=['Возможности']

pivot_clients

## Выигранные возможности

In [ ]:
clients.query('Статус == "Выиграна"')['Организация'].count()

In [ ]:
clients.query('Статус == "Выиграна"')['Организация']

### Выигранные возможности в разрезе услуг

In [ ]:
clients.query('Статус == "Выиграна"')['Услуга'].value_counts()

### Указанный источник запроса в выигранных возможностях

In [ ]:
clients.query('Статус == "Выиграна"')['Источник'].value_counts()

## Стоимость контракта 

Загрузим таблицу `price` с данными о стоимости контрактов и причине победы

In [ ]:
price = pd.read_excel('/Users/Olga/First attempt in Py/Site_rfp/contract_price_win.xlsx')
price.to_csv('price.csv', encoding='utf-8', index=False)
price.info()

In [ ]:
price

Объеденим список выигранных клиентов с данными о стоимости контракта

In [ ]:
win = clients.query('Статус == "Выиграна"')['Организация']

In [ ]:
opportunities_win = price.merge(win, on='Организация', how='left')
opportunities_win

### Сумма всех выигранных через обращение на сайта возможностей

In [ ]:
opportunities_win['Стоимость услуг'].sum()

### Средняя стоимость контракта

In [ ]:
opportunities_win['Стоимость услуг'].mean()

### Указаннная причина победы

In [ ]:
opportunities_win.pivot_table(index=['Причина'], aggfunc={'Организация': 'nunique'})

<a id="con"></a>
# Анализ конверсий 

### Количество возможностей в каждом месяце

В таблице с возможностями выделим стобец с месяцем создания возможности

In [ ]:
clients['Дата создания возможности'] = pd.to_datetime(clients['Создание'], format='%Y.%m.%d %H:%M:%S',infer_datetime_format=True)
clients['Месяц'] = clients['Дата создания возможности'].dt.month

In [ ]:
pivot_table_by_month_total_opportunities = clients.pivot_table(clients,index=['Месяц'], aggfunc={'Организация': 'nunique'}).reset_index()
pivot_table_by_month_total_opportunities.columns=['Месяц', 'Количество возможностей']
pivot_table_by_month_total_opportunities.sort_values(by = 'Месяц', ascending = True)

Загрузим таблицу `site_traffic_05_09_20` с данными о посещениях сайта для расчета конверсии 

In [ ]:
traffic = pd.read_excel('/Users/Olga/First attempt in Py/Site_rfp/site_traffic_05_09_20.xlsx')

traffic.to_csv('traffic.csv', encoding='utf-8', index=False)

traffic

Объеденим таблицу `pivot_table_by_month_total` с количеством обращений по месяцам с таблицей посещений по месяцам `traffic`

In [ ]:
conversion = traffic.merge(pivot_table_by_month_total, on='Месяц', how='left')

### Конверсия из посещений в запрос, % от общего количества посещений

In [ ]:
conversion['Конверсия в обращение, %'] = ((conversion['Количество обращений']/conversion['Посещений'])*100)
conversion


### Конверсия из обращения в возможность, % от общего количества запросов

In [ ]:
сonversion_to_opportunities = conversion.merge(pivot_table_by_month_total_opportunities, on='Месяц', how='left')
сonversion_to_opportunities.replace(np.nan, 0, inplace=True)
сonversion_to_opportunities['Конверсия в возможность, %'] = ((сonversion_to_opportunities['Количество возможностей']/сonversion_to_opportunities['Количество обращений'])*100).round(2)

сonversion_to_opportunities

### Количество выигранных возможностей в каждом месяце

In [ ]:
pivot_table_by_month_total_clients = clients.query('Статус == "Выиграна"').pivot_table(index=['Месяц'], aggfunc={'Организация': 'nunique'}).reset_index()
pivot_table_by_month_total_clients.columns=['Месяц', 'Количество выигранных возможностей']
pivot_table_by_month_total_clients.sort_values(by = 'Месяц', ascending = True)

### Конверсия в выигранную от общего количества обращений, %

In [ ]:
сonversion_opportunities_win = сonversion_to_opportunities.merge(pivot_table_by_month_total_clients, on='Месяц', how='left')

сonversion_opportunities_win['Конверсия в win, %'] = ((сonversion_opportunities_win['Количество выигранных возможностей']/сonversion_opportunities_win['Количество обращений'])*100).round(2)
сonversion_opportunities_win.replace(np.nan, 0, inplace=True)

## Итоговая таблица конверсий от общего количества обращений

In [ ]:
сonversion_opportunities_win

### График конверсий по месяцам

In [ ]:
usfull_colums = ['Конверсия в обращение, %', 'Конверсия в возможность, %','Конверсия в win, %']

In [ ]:
fig = px.line(сonversion_opportunities_win, x='Месяц', y=usfull_colums, title='Конверсии по месяцам',
             labels= dict(x = "Май, Июнь, Июль, Август, Сентябрь, Октябрь", y= "Процент конверсий"))
fig.show()

## Воронка конверсий из обращений в выигранные возможности по усредненным данным за 6 месяцев

In [ ]:
сonversion_opportunities_win.mean()

In [ ]:
fig = go.Figure(go.Funnel(
    y = ["Количество обращений", "Количество возможностей", "Количество выигранных возможностей"],
    x = [XX, XX, XX], textinfo = "value+percent initial"))

fig.update_layout(title={'text': "Воронка для усредненных данных за 6 месяцев"})

fig.show()

## Воронка конверсий из обращений в выигранные возможности помесячно¶

In [ ]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Май',
    y = ["Количество обращений", "Количество возможностей", "Количество выигранных возможностей"],
    x = [XX, XX, XX],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Июнь',
    orientation = "h",
    y = ["Количество обращений", "Количество возможностей", "Количество выигранных возможностей"],
    x = [XX, XX, XX],
    textposition = "outside",
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name = 'Июль',
    orientation = "h",
    y = ["Количество обращений", "Количество возможностей", "Количество выигранных возможностей"],
    x = [XX, XX, XX],
    textposition = "outside",
    textinfo = "value+percent total"))

fig.add_trace(go.Funnel(
    name = 'Август',
    orientation = "h",
    y = ["Количество обращений", "Количество возможностей", "Количество выигранных возможностей"],
    x = [XX, XX, XX],
    textposition = "outside",
    textinfo = "value+percent total"))

fig.add_trace(go.Funnel(
    name = 'Сентябрь',
    orientation = "h",
    y = ["Количество обращений", "Количество возможностей", "Количество выигранных возможностей"],
    x = [XX, XX, XX],
    textposition = "outside",
    textinfo = "value+percent total"))

fig.add_trace(go.Funnel(
    name = 'Октябрь',
    orientation = "h",
    y = ["Количество обращений", "Количество возможностей", "Количество выигранных возможностей"],
    x = [XX, XX, XX],
    textposition = "outside",
    textinfo = "value+percent total"))

fig.update_layout(title={'text': "Воронка помесячная"})

fig.show()
